In [1]:
!pip install transformers
!pip install accelerate datasets peft bitsandbytes

  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.9.2 requires torchaudio==2.7.0, which is not installed.
vllm 0.9.2 requires xformers==0.0.30; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
unsloth 2025.6.12 requires xformers>=0.0.27.post2, which is not installed.
vllm 0.9.2 requires setuptools<80,>=77.0.3; python_version > "3.11", but you have setuptools 80.9.0 which is incompatible.
vllm 0.9.2 requires torch==2.7.0, but you have torch 2.7.1+cu128 which is incompatible.
vllm 0.9.2 requires torchvision==0.22.0, but you have torchvision 0.22.1

In [2]:
import torch

In [3]:
from peft import LoraConfig
from transformers import AutoProcessor,BitsAndBytesConfig,Idefics2ForConditionalGeneration

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
DEVICE="cuda:0"
USE_LORA=False
USE_QLORA=True

In [5]:
processor=AutoProcessor.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    do_image_splitting=False
)

Chat templates should be in a 'chat_template.jinja' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.


In [ ]:
if USE_QLORA or USE_LORA:
    lora_config=LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules='.*(text_model|modality_projection|perceiver_resampler).*'
        use_dora=False if USE_QLORA else True,
        init_lora_weight='gaussian'
    )
    if USE_QLORA:
        bnb_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.float16
        )
    model=Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        quantization_config=bnb_config if USE_QLORA else None,
    ) 
    model.add_adapter(lora_config)
    model.enable_adapters() 
else:
    model=Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2" #A100 or H100
    ).to(DEVICE) 
         